In [1]:
import pandas as pd

# Load the CSV file
pathname = '/Users/jcoleman/Documents/--LARGE DATA--/#Pizzi/TBI-sham leak analysis/scripts/matlab-copy/September 2024/'
filename = 'analysisData_091724.csv'
df = pd.read_csv(pathname+filename)

print(df['filename'])
print(df['group'])

0     ZT10_16X6x_gg_800nm_m5_roi2_00003_ROIs_classA
1     ZT10_16X6x_gg_800nm_m6_roi1_00001_ROIs_classA
2     ZT10_16X6x_gg_800nm_m7_roi1_00001_ROIs_classA
3     ZT10_16X6x_gg_800nm_m9_roi1_00002_ROIs_classA
4    ZT10_16X6x_gg_800nm_m10_roi1_00001_ROIs_classA
5    ZT10_16X6x_gg_800nm_m11_roi1_00001_ROIs_classA
6    ZT10_16X6x_gg_800nm_m12_roi1_00001_ROIs_classA
7    ZT10_16X6x_gg_800nm_m13_roi1_00002_ROIs_classA
8    ZT10_16X6x_gg_800nm_m15_roi1_00001_ROIs_classA
9    ZT10_16X6x_gg_800nm_m16_roi1_00001_ROIs_classA
Name: filename, dtype: object
0    A
1    A
2    B
3    A
4    A
5    A
6    A
7    B
8    B
9    B
Name: group, dtype: object


In [2]:
# dfTable = pd.read_table(pathname+filename)

# print(dfTable.head())

In [4]:
# Transfrom CSV from allData (MATLAB output)
# import pandas as pd
import numpy as np

"""
Convention and column order: It's generally better to keep the 'rowID' as the first column across all output files for consistency.
This makes it easier to identify and work with the data, especially when doing analyses or merging datasets later.
Data format: The format we've created is indeed considered "long" or "tidy" format.

In tidy data:
Each variable forms a column
Each observation forms a row
Each type of observational unit forms a table

This format is preferred for many types of analyses and is especially useful for tools like ggplot in R
or seaborn in Python.
"""

# Read the input CSV file
# input_file = 'sampleData2_input.csv'
# df = pd.read_csv(input_file)

# Assuming maximum 40 measurements per row per metric/set of values
# Set the max number of 'rois' (ie data values per metric per observation-subject in the dataset)
max_roi_num = 40

headers_to_extract = ['meanFWHM', 'stdevFWHM', 'minFWHM', 'maxFWHM', 'meanFeFv', 'stdevFeFv', 'minFeFv', 'maxFeFv']
# TESTheaders_to_extract = ['meanFWHM', 'stdevFWHM', 'minFWHM', 'maxFWHM']
logicals_to_extract = [
    'capLogical',
    'venLogical',
    'artLogical',
    'midLogical',
    'preLogical',
    'pstLogical',
    'midCapLogical',
    'preCapLogical',
    'pstCapLogical',
    'midVenLogical',
    'preVenLogical',
    'pstVenLogical',
    'midArtLogical',
    'preArtLogical',
    'pstArtLogical',
]
# logicals_to_extract = ['capLogical', 'venLogical', 'artLogical']
# logicals_to_extract = ['capLogical', 'venLogical']

# Function to create the rowID
def create_row_id(row, index):
    return f"{row['filename']}_{index:02d}"

# Function to extract columns and create new rows
def extract_columns(row):
    new_rows = []
    for i in range(1, max_roi_num): # Set maximum measurements per row per metric per set of values (ie rois analyzed per subject)
        new_row = {'rowID': create_row_id(row, i)}
        for header in headers_to_extract:
            new_row[header] = row.get(f"{header}_{i}", np.nan)
        for logical in logicals_to_extract:
            new_row[logical] = row.get(f"{logical}_{i}", np.nan)
        new_row['group'] = row['group']
        new_rows.append(new_row)
    return new_rows

# Create the new dataframe
new_rows = [row for rows in df.apply(extract_columns, axis=1) for row in rows]
df_output = pd.DataFrame(new_rows)

# Ensure 'rowID' is the first column
column_order = ['rowID'] + [col for col in df_output.columns if col != 'rowID']
df_output = df_output[column_order]

# Create output1a (with all rows, including those with NaN values)
df_output1a = df_output.copy()
df_output1a.to_csv('Data1_output1a_transii.csv', index=False)

# Create output1b (without rows that have all NaN values in headers_to_extract)
df_output1b = df_output[df_output[headers_to_extract].notna().any(axis=1)]
df_output1b.to_csv('Data1_output1b_transii.csv', index=False)

# Create output1c (without logicals and group, and without rows that have all NaN values in headers_to_extract)
df_output1c = df_output[['rowID'] + headers_to_extract]
df_output1c = df_output1c[df_output1c[headers_to_extract].notna().any(axis=1)]
df_output1c.to_csv('Data1_output1c_transii.csv', index=False)

print("Transformation complete. Output files have been generated.")
print("Be sure to delete all rows with empty cells/nans before PCA.")

Transformation complete. Output files have been generated.
Be sure to delete all rows with empty cells/nans before PCA.


In [5]:
print(df_output1c.head())

print(df_output1a.shape)
print(df_output1b.shape)
print(df_output1c.shape)

                                               rowID  meanFWHM  stdevFWHM  \
0   ZT10_16X6x_gg_800nm_m5_roi2_00003_ROIs_classA_01  3.740378   0.120027   
1   ZT10_16X6x_gg_800nm_m5_roi2_00003_ROIs_classA_02  4.100067   0.120158   
2   ZT10_16X6x_gg_800nm_m5_roi2_00003_ROIs_classA_03  6.326261   0.091814   
3   ZT10_16X6x_gg_800nm_m5_roi2_00003_ROIs_classA_04  6.066305   0.070735   
39  ZT10_16X6x_gg_800nm_m6_roi1_00001_ROIs_classA_01  5.886713   0.134211   

     minFWHM   maxFWHM  meanFeFv  stdevFeFv   minFeFv   maxFeFv  
0   3.542297  3.907355  0.301351   0.088444  0.164055  0.406216  
1   3.922870  4.245508  0.718676   0.103000  0.559359  0.933077  
2   6.202881  6.441604  0.083048   0.026864  0.049834  0.148631  
3   5.974274  6.145408  0.849531   0.084406  0.713789  0.964376  
39  5.693535  6.101866  0.222588   0.057162  0.110756  0.307333  
(390, 25)
(192, 25)
(192, 9)


In [6]:
df_pca_withIDs = df_output1b.dropna(how='any')
df_pca_withIDs.to_csv('df_pca_withIDs.csv', index=False)
print(df_pca_withIDs.head())
print(df_pca_withIDs.shape)

                                               rowID  meanFWHM  stdevFWHM  \
0   ZT10_16X6x_gg_800nm_m5_roi2_00003_ROIs_classA_01  3.740378   0.120027   
1   ZT10_16X6x_gg_800nm_m5_roi2_00003_ROIs_classA_02  4.100067   0.120158   
2   ZT10_16X6x_gg_800nm_m5_roi2_00003_ROIs_classA_03  6.326261   0.091814   
3   ZT10_16X6x_gg_800nm_m5_roi2_00003_ROIs_classA_04  6.066305   0.070735   
39  ZT10_16X6x_gg_800nm_m6_roi1_00001_ROIs_classA_01  5.886713   0.134211   

     minFWHM   maxFWHM  meanFeFv  stdevFeFv   minFeFv   maxFeFv  capLogical  \
0   3.542297  3.907355  0.301351   0.088444  0.164055  0.406216         1.0   
1   3.922870  4.245508  0.718676   0.103000  0.559359  0.933077         1.0   
2   6.202881  6.441604  0.083048   0.026864  0.049834  0.148631         1.0   
3   5.974274  6.145408  0.849531   0.084406  0.713789  0.964376         1.0   
39  5.693535  6.101866  0.222588   0.057162  0.110756  0.307333         1.0   

    ...  midCapLogical  preCapLogical  pstCapLogical  midVenLo

In [7]:
df_for_pca_temp = df_output1c.dropna(how='any')
# df_for_pca_clean = df_for_pca_temp[df_for_pca_temp['rowID'] != 'row_to_ignore']
df_for_pca_clean  = df_for_pca_temp.drop('rowID', axis=1)

df_for_pca_clean.to_csv('df_for_pca_clean.csv', index=False)
print(df_for_pca_clean.head())
print(df_for_pca_temp.shape)
print(df_for_pca_temp['rowID'].dtype)
print(df_for_pca_clean.shape)

    meanFWHM  stdevFWHM   minFWHM   maxFWHM  meanFeFv  stdevFeFv   minFeFv  \
0   3.740378   0.120027  3.542297  3.907355  0.301351   0.088444  0.164055   
1   4.100067   0.120158  3.922870  4.245508  0.718676   0.103000  0.559359   
2   6.326261   0.091814  6.202881  6.441604  0.083048   0.026864  0.049834   
3   6.066305   0.070735  5.974274  6.145408  0.849531   0.084406  0.713789   
39  5.886713   0.134211  5.693535  6.101866  0.222588   0.057162  0.110756   

     maxFeFv  
0   0.406216  
1   0.933077  
2   0.148631  
3   0.964376  
39  0.307333  
(190, 9)
object
(190, 8)


In [8]:
# print(df_for_pca_temp['rowID'].head())  # Show first few values
# print(df_for_pca_temp['rowID'].unique())  # Show unique values
print(df_for_pca_temp['rowID'].value_counts())  # Show value counts

ZT10_16X6x_gg_800nm_m5_roi2_00003_ROIs_classA_01     1
ZT10_16X6x_gg_800nm_m13_roi1_00002_ROIs_classA_07    1
ZT10_16X6x_gg_800nm_m12_roi1_00001_ROIs_classA_35    1
ZT10_16X6x_gg_800nm_m12_roi1_00001_ROIs_classA_36    1
ZT10_16X6x_gg_800nm_m12_roi1_00001_ROIs_classA_37    1
                                                    ..
ZT10_16X6x_gg_800nm_m11_roi1_00001_ROIs_classA_08    1
ZT10_16X6x_gg_800nm_m11_roi1_00001_ROIs_classA_09    1
ZT10_16X6x_gg_800nm_m11_roi1_00001_ROIs_classA_10    1
ZT10_16X6x_gg_800nm_m11_roi1_00001_ROIs_classA_11    1
ZT10_16X6x_gg_800nm_m16_roi1_00001_ROIs_classA_18    1
Name: rowID, Length: 190, dtype: int64


In [9]:
# Now, let's separate the PCA analysis
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

print("\n--- PCA Analysis ---")

# Scale the data
scaler = StandardScaler()
df_scaled = pd.DataFrame(
    scaler.fit_transform(df_for_pca_clean), 
    columns=df_for_pca_clean.columns, 
    index=df_for_pca_clean.index
)

print(df_scaled.head())
print(df_scaled.shape)


--- PCA Analysis ---
    meanFWHM  stdevFWHM   minFWHM   maxFWHM  meanFeFv  stdevFeFv   minFeFv  \
0  -0.753129  -0.264182 -0.734643 -0.770704 -0.957662  -0.428220 -1.055814   
1  -0.552381  -0.263166 -0.520402 -0.585311 -0.241254  -0.335022 -0.180405   
2   0.690091  -0.481656  0.763115  0.618701 -1.332417  -0.822510 -1.308759   
3   0.545006  -0.644145  0.634423  0.456311 -0.016618  -0.454078  0.161582   
39  0.444773  -0.154842  0.476382  0.432439 -1.092873  -0.628519 -1.173847   

     maxFeFv  
0  -0.929415  
1  -0.250892  
2  -1.261148  
3  -0.210584  
39 -1.056762  
(190, 8)


In [10]:
# Perform PCA
pca = PCA()
pca_result = pca.fit_transform(df_scaled)

# Create a DataFrame with PCA results
pca_df = pd.DataFrame(
    data=pca_result,
    columns=[f'PC{i+1}' for i in range(pca_result.shape[1])],
    index=df_for_pca_clean.index
)

print("\nPCA Results:")
print(pca_df.head())


PCA Results:
         PC1       PC2       PC3       PC4       PC5       PC6       PC7  \
0  -0.942118 -1.931356  0.448974  0.159236 -0.058432 -0.039371 -0.006136   
1  -0.037306 -1.109899 -0.102672 -0.086258 -0.002619  0.044717  0.007253   
2  -2.704731 -0.062154  0.071763  0.320889 -0.022847  0.013285 -0.002027   
3  -0.734230  0.592860 -0.840275  0.042960 -0.003039 -0.027507 -0.004825   
39 -2.130028 -0.193071  0.331685  0.174882 -0.067659  0.012554 -0.012839   

         PC8  
0   0.007993  
1  -0.020397  
2  -0.017319  
3  -0.017624  
39  0.004147  


In [11]:
# Calculate and print explained variance ratio
explained_variance_ratio = pca.explained_variance_ratio_
print("\nExplained Variance Ratio:")
for i, ratio in enumerate(explained_variance_ratio):
    print(f"PC{i+1}: {ratio:.4f}")

# Cumulative explained variance
cumulative_variance_ratio = np.cumsum(explained_variance_ratio)
print("\nCumulative Explained Variance Ratio:")
for i, ratio in enumerate(cumulative_variance_ratio):
    print(f"PC1 to PC{i+1}: {ratio:.4f}")


Explained Variance Ratio:
PC1: 0.4381
PC2: 0.3601
PC3: 0.1378
PC4: 0.0611
PC5: 0.0019
PC6: 0.0007
PC7: 0.0001
PC8: 0.0001

Cumulative Explained Variance Ratio:
PC1 to PC1: 0.4381
PC1 to PC2: 0.7982
PC1 to PC3: 0.9361
PC1 to PC4: 0.9972
PC1 to PC5: 0.9991
PC1 to PC6: 0.9998
PC1 to PC7: 0.9999
PC1 to PC8: 1.0000
